In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.metrics import classification_report

#### Loading the Data

In [2]:
df = pd.read_csv("Bank_Personal_Loan_Modelling.csv")

In [3]:
print("Shape : ", df.shape)
df.head()

Shape :  (5000, 14)


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


#### Data Cleaning and Feature Engineering

In [4]:
# Deleting Columns which are not necessary
df.drop(["ID", "ZIP Code"],axis=1,inplace=True)

In [5]:
print("Duplicate : ", df.duplicated().sum())
print("Null : ", df.isnull().sum().sum())

Duplicate :  13
Null :  0


In [6]:
print(df["Experience"].unique())
df["Experience"] = abs(df["Experience"])

[ 1 19 15  9  8 13 27 24 10 39  5 23 32 41 30 14 18 21 28 31 11 16 20 35
  6 25  7 12 26 37 17  2 36 29  3 22 -1 34  0 38 40 33  4 -2 42 -3 43]


In [7]:
df.columns

Index(['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education',
       'Mortgage', 'Personal Loan', 'Securities Account', 'CD Account',
       'Online', 'CreditCard'],
      dtype='object')

In [8]:
df = df[['Age', 'Experience', 'Income', 'Family', 'CCAvg','Education', 'Mortgage', 'Securities Account','CD Account', 'Online', 'CreditCard', 'Personal Loan']]

In [9]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

### Train Test Split

In [10]:
x,y = X,Y

In [11]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=69)

In [12]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [13]:
x_train.shape, y_train.shape

((3750, 11), (3750,))

# PyTorch 

In [14]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [15]:
BATCH_SIZE = 1

In [16]:
train_x = torch.from_numpy(x_train).to(torch.float32)
train_y = torch.from_numpy(y_train).to(torch.float32)

In [17]:
train_x.shape, train_y.shape

(torch.Size([3750, 11]), torch.Size([3750]))

In [18]:
data = TensorDataset(train_x,train_y)
data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)

## Building Model

In [19]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        
        self.layer1 = torch.nn.Linear(11,16)
        self.layer2 = torch.nn.Linear(16,1)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

In [20]:
model = Model()
# model.to(device)
print(model)

Model(
  (layer1): Linear(in_features=11, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)


# Weight Optimization using Genetic Algorithm

In [21]:
# Accuracy 
def fitness_function(model):
    
    y_pred = model(train_x)
    y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
    accuracy = (y_pred == train_y).sum().float().item() / len(data.dataset)
    
    return accuracy

In [22]:
# Crossover and Mutation
def crossover_mutation(model1,model2):
    # Shape of the weights and biases
    shp = [i.numpy().shape for i in model1.parameters()]
    sz = [i[0]*i[1] if len(i) == 2 else i[0] for i in shp]
    
    # Flattening the parameters for cross over
    matrix_1 = np.concatenate([i.numpy().flatten() for i in model1.parameters()])
    matrix_2 = np.concatenate([i.numpy().flatten() for i in model2.parameters()])
    
    # Crossover
    start = len(matrix_1)//2 - 10
    end = len(matrix_1)//2 + 10
    mid_value = random.randrange(start,end)
    
    main_child_1 = np.concatenate([matrix_1[:mid_value], matrix_2[mid_value:]])
    main_child_2 = np.concatenate([matrix_2[:mid_value], matrix_1[mid_value:]])
    
    # Child 1 Mutation
    random_start = random.randrange(0,len(matrix_1)//2)
    random_end = random.randrange(random_start,len(matrix_1))
    
    child_1_mutate = main_child_1.copy()
    child_1_mutate[random_start:random_end] = child_1_mutate[random_start:random_end][::-1]
    
    # Child 2 Mutation
    random_start = random.randrange(0,len(matrix_1)//2)
    random_end = random.randrange(random_start,len(matrix_1))
    
    child_2_mutate = main_child_2.copy()
    child_2_mutate[random_start:random_end] = child_2_mutate[random_start:random_end][::-1]
    
    # Converting the array to parameters
    children = [main_child_1, main_child_2, child_1_mutate, child_2_mutate]
    output = list()
    
    for child in children:
        param = list()
        cum_sum = 0
        for i in range(len(sz)):
            array = child[cum_sum : cum_sum + sz[i]]
            array = array.reshape(shp[i])
            cum_sum += sz[i]
            param.append(array)
        param = np.array(param, dtype="object")
        output.append(param)
    
    output = np.array(output, dtype="object")
    return output

In [23]:
# Training 

torch.manual_seed(69)
torch.set_grad_enabled(False)
population_size = 10

def train(no):
    # Initial Population
    population = np.array([Model() for i in range(population_size)])
    
    best_model = None
    
    for loop in range(no):
        # Sort population based up on fitness function
        population = population[np.argsort([fitness_function(model) for model in population])]
        
        # Printing Max Accuracy
        best_model = population[-1]
        if(loop%10 == 0):
            print("Gen", loop, " :", fitness_function(population[-1]))
        
        # Parent Selection
        parent_max_1 = population[-1]
        parent_max_2 = population[-2]
        parent_max_3 = population[-3]
        parent_max_4 = population[-4] 
        parent_1 = population[0]
        parent_2 = population[1]
        
        # Crossover
        output_1 = crossover_mutation(parent_max_1, parent_max_3)
        output_2 = crossover_mutation(parent_max_2, parent_max_4)
        output = np.concatenate([output_1, output_2])
        
        # New population
        new_population = np.array([Model() for i in range(len(output))])
        for count, model in enumerate(new_population, 0):
            for index, param in enumerate(model.parameters(), 0):
                param.data = (torch.tensor(output[count][index]))
        
        # Add Poor performing parent to the new population
        new_population = np.concatenate([new_population, [parent_1, parent_2]])
        
        # New population becomes your next population
        population = new_population.copy()
        
    return best_model

In [24]:
best_model = train(100)

Gen 0  : 0.824
Gen 10  : 0.9082666666666667
Gen 20  : 0.9264
Gen 30  : 0.9250666666666667
Gen 40  : 0.9314666666666667
Gen 50  : 0.9317333333333333
Gen 60  : 0.9317333333333333
Gen 70  : 0.9328
Gen 80  : 0.9338666666666666
Gen 90  : 0.9357333333333333


## Testing 

In [25]:
test_x = torch.from_numpy(x_test).to(torch.float32)
test_y = torch.from_numpy(y_test).to(torch.float32)

In [26]:
test = TensorDataset(test_x,test_y)
test = DataLoader(test,batch_size=1)

### Classification Report

In [27]:
y_pred = best_model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
print(classification_report(y_pred,test_y))

              precision    recall  f1-score   support

           0       0.99      0.94      0.97      1202
           1       0.37      0.88      0.52        48

    accuracy                           0.94      1250
   macro avg       0.68      0.91      0.74      1250
weighted avg       0.97      0.94      0.95      1250

